In [16]:
import numpy as np
from __future__ import print_function
import numpy as np
import scipy
from matplotlib import pyplot as plt
import csv
from sklearn import metrics
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.linear_model import OrthogonalMatchingPursuitCV
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn import metrics
from sklearn.preprocessing import normalize
from scipy.sparse import coo_matrix
from sklearn.utils import resample
import time
from sklearn.cluster import KMeans
from scipy.stats import pearsonr
from sklearn.cluster import AgglomerativeClustering


In [8]:
D = np.load("D.npy")

In [21]:
D_test = np.load("dict_sub3_l1b.npy")

In [10]:
D.shape
D1 = np.transpose(D)
D1.shape

(1080, 240)

In [17]:
def calculate_accuracy(Xts, yts, D, minimum):
    y_pred1 = []
    y_pred2 = []
    y_pred3 = []
    y_pred4 = []
    diff = []
    class1_avg = 0
    class2_avg = 0
    class3_avg = 0
    class4_avg = 0
    counter1 = 0
    
    #print(D.shape)
    
    for i in range(len(Xts)):
        features = Xts[i]

        omp = OrthogonalMatchingPursuit(n_nonzero_coefs=30)
        omp.fit(D, features)
        coef = omp.coef_

        s1 = 0
        s2 = 0
        s3 = 0
        s4 = 0

        l1 = 0
        l2 = 0
        l3 = 0
        l4 = 0

        a1 = 0
        a2 = 0
        a3 = 0
        a4 = 0

        list1 = coef[0:minimum]
        list2 = coef[minimum:2 * minimum]
        list3 = coef[2 * minimum:3 * minimum]
        list4 = coef[3 * minimum:4 * minimum]

        c1 = (sum(z * z for z in list1))**(1 / 2.0)
        c2 = (sum(z * z for z in list2))**(1 / 2.0)
        c3 = (sum(z * z for z in list3))**(1 / 2.0)
        c4 = (sum(z * z for z in list4))**(1 / 2.0)

        s1 = np.std(list1)
        s2 = np.std(list2)
        s3 = np.std(list3)
        s4 = np.std(list4)

        a1 = max(list1)
        a2 = max(list2)
        a3 = max(list3)
        a4 = max(list4)

        for i1 in range(minimum):
            l1 = l1 + coef[i1]

        for i1 in xrange(minimum, 2 * minimum):
            l2 = l2 + coef[i1]

        for i1 in range(2 * minimum, 3 * minimum):
            l3 = l3 + coef[i1]

        for i1 in xrange(3 * minimum, 4 * minimum):
            l4 = l4 + coef[i1]

        if s1 >= s2 and s1 >= s3 and s1 >= s4:
            y_pred1.append(0)
        elif s2 >= s1 and s2 >= s3 and s2 >= s4:
            y_pred1.append(1)
        elif s3 >= s1 and s3 >= s2 and s3 >= s4:
            y_pred1.append(2)
        else:
            y_pred1.append(3)
            if (yts[i] == 2):
                if (counter1 == 0):
                    counter1 += 1
                    idx_r, = coef.nonzero()

        if l1 >= l2 and l1 >= l3 and l1 >= l4:
            y_pred2.append(0)
        elif l2 >= l1 and l2 >= l3 and l2 >= l4:
            y_pred2.append(1)
        elif l3 >= l1 and l3 >= l2 and l3 >= l4:
            y_pred2.append(2)
        else:
            y_pred2.append(3)

        if a1 >= a2 and a1 >= a3 and a1 >= a4:
            y_pred3.append(0)
        elif a2 >= a1 and a2 >= a3 and a2 >= a4:
            y_pred3.append(1)
        elif a3 >= a1 and a3 >= a2 and a3 >= a4:
            y_pred3.append(2)
        else:
            y_pred3.append(3)

        if c1 >= c2 and c1 >= c3 and c1 >= c4:
            y_pred4.append(0)
        elif c2 >= c1 and c2 >= c3 and c2 >= c4:
            y_pred4.append(1)
        elif c3 >= c1 and c3 >= c2 and c3 >= c4:
            y_pred4.append(2)
        else:
            y_pred4.append(3)
    
    class1_avg += metrics.accuracy_score(
        yts, y_pred4, normalize=True, sample_weight=None)
    class2_avg += metrics.accuracy_score(
        yts, y_pred3, normalize=True, sample_weight=None)
    class3_avg += metrics.accuracy_score(
        yts, y_pred1, normalize=True, sample_weight=None)
    class4_avg += metrics.accuracy_score(
        yts, y_pred2, normalize=True, sample_weight=None)

    return y_pred4, y_pred3, y_pred1, y_pred2, class1_avg, class2_avg, class3_avg, class4_avg

In [22]:
r = 600
y_test3 = np.zeros((r*4,))
for i in range(r*4):
    if i<r: y_test3[i]=0
    elif i<r*2: y_test3[i]=1
    elif i<r*3: y_test3[i]=2
    else: y_test3[i]=3
y_test3.shape

(2400,)

In [23]:
# dict = sub1  test cases are of sub1
y_classifier1, y_classifier2, y_classifier3, y_classifier4, a1, a2, a3, a4 = calculate_accuracy(
            D_test, y_test3.reshape(-1,1), np.transpose(D1), 1080/4)
print((a1+a2+a3+a4) / 4)
print('class1:', a1)
print('class2:', a2)
print('class3:', a3)
print('class4:', a4)

0.928645833333
class1: 0.931666666667
class2: 0.93125
class3: 0.93125
class4: 0.920416666667
